# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [5]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [6]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [7]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [8]:
# Load the historical closing prices for bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [9]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [10]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [11]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [12]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 1

# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [13]:
# Use 70% of the data for training and the remaineder for testing
split = 0.7 * len(X)
X_train = X[:int(split - 1)]
X_test = X[int(split):]
y_train = y[:int(split - 1)]
y_test = y[int(split):]

In [14]:
# Use MinMaxScaler to scale the data between 0 and 1. 
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()

# data
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# labels
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [15]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [17]:
# Build the LSTM model. 

model = Sequential()
model.add(LSTM(
    units=40, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))

model.add(Dropout(0.2))
model.add(LSTM(units=40, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=40))
model.add(Dropout(0.2))
model.add(Dense(1))

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 40)             6720      
_________________________________________________________________
dropout (Dropout)            (None, 1, 40)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 40)             12960     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 40)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 4

In [20]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train, y_train, shuffle=False, epochs=15, verbose=1, batch_size=3)

Train on 377 samples
Epoch 1/15
377/377 [==============================] - 24s 63ms/sample - loss: 0.0858
Epoch 2/15
377/377 [==============================] - 4s 10ms/sample - loss: 0.0378
Epoch 3/15
377/377 [==============================] - 4s 9ms/sample - loss: 0.0279
Epoch 4/15
377/377 [==============================] - 4s 9ms/sample - loss: 0.0208
Epoch 5/15
377/377 [==============================] - 4s 10ms/sample - loss: 0.0120
Epoch 6/15
377/377 [==============================] - 4s 10ms/sample - loss: 0.0057
Epoch 7/15
377/377 [==============================] - 4s 10ms/sample - loss: 0.0037
Epoch 8/15
377/377 [==============================] - 4s 10ms/sample - loss: 0.0035
Epoch 9/15
377/377 [==============================] - 4s 11ms/sample - loss: 0.0033
Epoch 10/15
377/377 [==============================] - 5s 13ms/sample - loss: 0.0033
Epoch 11/15
377/377 [==============================] - 5s 13ms/sample - loss: 0.00340s - loss: 0.00
Epoch 12/15
377/377 [==================

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [21]:
# Evaluate the model
model.evaluate(X_test, y_test)

163/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.0049751433800974505

In [22]:
# Make some predictions
predictions = model.predict(X_test)

In [23]:
# Recover the original prices instead of the scaled version
predicted_price = scaler.inverse_transform(predictions)
real_price = scaler.inverse_transform(y_test.reshape(-1, 1))

In [24]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_price.ravel(),
    "Predicted": predicted_price.ravel()
})
stocks.head()

,Real,Predicted
0,3670.919922,3571.499268
1,3670.919922,3616.113770
2,3912.570068,3616.113770
3,3924.239990,3818.047607
4,3974.050049,3827.843994


In [25]:
# Plot the real vs predicted values as a line chart
stocks.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)